In [1]:
import numpy as np
import pandas as pd
import importlib.resources
import phoenix as phx
import os
import openai
import json
import re
from dotenv import load_dotenv
import warnings 
warnings.filterwarnings('ignore')
load_dotenv()

True

In [2]:
path = importlib.resources.files('phoenix')
sepsis = pd.read_csv(path.joinpath('data').joinpath('sepsis.csv'))
sepsis.head()

,pid,age,fio2,pao2,spo2,vent,gcs_total,pupil,platelets,inr,...,epinephrine,milrinone,norepinephrine,vasopressin,glucose,anc,alc,creatinine,bilirubin,alt
0,1,0.06,0.75,NaN,99.0,1,NaN,NaN,199.0,1.46,...,1,1,0,0,NaN,NaN,NaN,1.03,NaN,36.0
1,2,201.70,0.75,75.3,95.0,1,5.0,both-reactive,243.0,1.18,...,0,0,1,0,110.0,14.220,2.220,0.51,0.2,32.0
2,3,20.80,1.00,49.5,NaN,1,15.0,both-reactive,49.0,1.60,...,0,0,0,0,93.0,2.210,0.190,0.33,0.8,182.0
3,4,192.50,NaN,NaN,NaN,0,14.0,NaN,NaN,1.30,...,0,0,0,0,110.0,3.184,0.645,0.31,8.5,21.0
4,5,214.40,NaN,38.7,95.0,0,NaN,NaN,393.0,NaN,...,0,0,0,0,NaN,NaN,NaN,0.52,NaN,NaN


In [3]:
sepsis.columns

Index(['pid', 'age', 'fio2', 'pao2', 'spo2', 'vent', 'gcs_total', 'pupil',
       'platelets', 'inr', 'd_dimer', 'fibrinogen', 'dbp', 'sbp', 'lactate',
       'dobutamine', 'dopamine', 'epinephrine', 'milrinone', 'norepinephrine',
       'vasopressin', 'glucose', 'anc', 'alc', 'creatinine', 'bilirubin',
       'alt'],
      dtype='object')

In [30]:
#Respiratory Dysfunction Scoring
resp = phx.phoenix_respiratory(
                                  pf_ratio = sepsis["pao2"] / sepsis["fio2"],
                                  sf_ratio = np.where(sepsis["spo2"] <= 97, sepsis["spo2"] / sepsis["fio2"], np.nan),
                                  imv      = sepsis["vent"],
                                  other_respiratory_support = (sepsis["fio2"] > 0.21).astype(int).to_numpy()
    )
print(f"Respiratory Dysfunction Scores: {resp}",f", array lenght : {resp.shape[0]}")
#Cardiovascular Dysfunction Scoring
card = phx.phoenix_cardiovascular(
                                  vasoactives = sepsis["dobutamine"] + sepsis["dopamine"] + sepsis["epinephrine"] + sepsis["milrinone"] + sepsis["norepinephrine"] + sepsis["vasopressin"],
                                  lactate = sepsis["lactate"],
                                  age = sepsis["age"],
                                  map = sepsis["dbp"] + (sepsis["sbp"] - sepsis["dbp"]) / 3
                                )
print(f"Cardiovascular Dysfunction Scores: {card}",f", array lenght : {card.shape[0]}")
#Coagulation Dysfunction Scoring
coag = phx.phoenix_coagulation(
                                platelets = sepsis['platelets'],
                                inr = sepsis['inr'],
                                d_dimer = sepsis['d_dimer'],
                                fibrinogen = sepsis['fibrinogen']
                              )
print(f"Coagulation Dysfunction Scores: {coag}", f", array lenght : {coag.shape[0]}")
#Neurologic Dysfunction Scoring
neuro = phx.phoenix_neurologic(
                                gcs = sepsis["gcs_total"],
                                fixed_pupils = (sepsis["pupil"] == "both-fixed").astype(int)
                              )
print(f"Neurologic Dysfunction Scores: {neuro}", f", array lenght : {neuro.shape[0]}")
#Endocrine Dysfunction Scoring
endo = phx.phoenix_endocrine(sepsis["glucose"])
print(f"Endocrine Dysfunction Scores: {endo}", f", array lenght : {endo.shape[0]}")
#Immunologic Dysfunction Scoring
immu = phx.phoenix_immunologic(sepsis["anc"], sepsis["alc"])
print(f"Immunologic Dysfunction Scores: {immu}", f", array lenght : {immu.shape[0]}")
# Renal Dysfunction Scoring
renal = phx.phoenix_renal(sepsis["creatinine"], sepsis["age"])
print(f"Renal Dysfunction Scores: {renal}", f", array lenght : {renal.shape[0]}")
#Hepatic Dysfunction Scoring
hepatic = phx.phoenix_hepatic(sepsis["bilirubin"], sepsis["alt"])
print(f"Hepatic Dysfunction Scores: {hepatic}", f", array lenght : {hepatic.shape[0]}")


Respiratory Dysfunction Scores: [0 3 3 0 0 3 3 0 3 3 3 1 0 2 3 0 2 3 2 0] , array lenght : 20
Cardiovascular Dysfunction Scores: [2 2 1 0 0 1 4 0 3 0 3 0 0 2 3 2 2 2 2 1] , array lenght : 20
Coagulation Dysfunction Scores: [1 1 2 1 0 2 2 1 1 0 1 0 0 1 2 1 1 2 0 1] , array lenght : 20
Neurologic Dysfunction Scores: [0 1 0 0 0 1 0 0 1 1 2 0 0 0 0 0 0 0 0 0] , array lenght : 20
Endocrine Dysfunction Scores: [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1] , array lenght : 20
Immunologic Dysfunction Scores: [0 1 1 1 0 1 0 1 1 1 0 0 0 0 0 1 1 0 1 1] , array lenght : 20
Renal Dysfunction Scores: [1 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0] , array lenght : 20
Hepatic Dysfunction Scores: [0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0] , array lenght : 20


In [31]:
phoenix = phx.phoenix(
                            pf_ratio = sepsis["pao2"] / sepsis["fio2"],
                            sf_ratio = sepsis["spo2"] / sepsis["fio2"],
                            imv      = sepsis["vent"],
                            other_respiratory_support = (sepsis["fio2"] > 0.21).astype(int).to_numpy(),
                            vasoactives = sepsis["dobutamine"] + sepsis["dopamine"] + sepsis["epinephrine"] + sepsis["milrinone"] + sepsis["norepinephrine"] + sepsis["vasopressin"],
                            lactate = sepsis["lactate"],
                            age = sepsis["age"],
                            map = sepsis["dbp"] + (sepsis["sbp"] - sepsis["dbp"]) / 3,
                            platelets = sepsis['platelets'],
                            inr = sepsis['inr'],
                            d_dimer = sepsis['d_dimer'],
                            fibrinogen = sepsis['fibrinogen'],
                            gcs = sepsis["gcs_total"],
                            fixed_pupils = (sepsis["pupil"] == "both-fixed").astype(int))
phoenix

,phoenix_respiratory_score,phoenix_cardiovascular_score,phoenix_coagulation_score,phoenix_neurologic_score,phoenix_sepsis_score,phoenix_sepsis,phoenix_septic_shock
0,3,2,1,0,6,1,1
1,3,2,1,1,7,1,1
2,3,1,2,0,6,1,1
3,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0
5,3,1,2,1,7,1,1
6,3,4,2,0,9,1,1
7,0,0,1,0,1,0,0
8,3,3,1,1,8,1,1
9,3,0,0,1,4,1,0


In [32]:
phoenix8_scores = phx.phoenix8(
    pf_ratio = sepsis["pao2"] / sepsis["fio2"],
    sf_ratio = np.where(sepsis["spo2"] <= 97, sepsis["spo2"] / sepsis["fio2"], np.nan),
    imv      = sepsis["vent"],
    other_respiratory_support = (sepsis["fio2"] > 0.21).astype(int).to_numpy(),
    vasoactives = sepsis["dobutamine"] + sepsis["dopamine"] + sepsis["epinephrine"] + sepsis["milrinone"] + sepsis["norepinephrine"] + sepsis["vasopressin"],
    lactate = sepsis["lactate"],
    map = sepsis["dbp"] + (sepsis["sbp"] - sepsis["dbp"]) / 3,
    platelets = sepsis['platelets'],
    inr = sepsis['inr'],
    d_dimer = sepsis['d_dimer'],
    fibrinogen = sepsis['fibrinogen'],
    gcs = sepsis["gcs_total"],
    fixed_pupils = (sepsis["pupil"] == "both-fixed").astype(int),
    glucose = sepsis["glucose"],
    anc = sepsis["anc"],
    alc = sepsis["alc"],
    creatinine = sepsis["creatinine"],
    bilirubin = sepsis["bilirubin"],
    alt = sepsis["alt"],
    age = sepsis["age"])
phoenix8_scores

,phoenix_respiratory_score,phoenix_cardiovascular_score,phoenix_coagulation_score,phoenix_neurologic_score,phoenix_sepsis_score,phoenix_sepsis,phoenix_septic_shock,phoenix_endocrine_score,phoenix_immunologic_score,phoenix_renal_score,phoenix_hepatic_score,phoenix8_score
0,0,2,1,0,3,1,1,0,0,1,0,4
1,3,2,1,1,7,1,1,0,1,0,0,8
2,3,1,2,0,6,1,1,0,1,0,1,8
3,0,0,1,0,1,0,0,0,1,0,1,3
4,0,0,0,0,0,0,0,0,0,0,0,0
5,3,1,2,1,7,1,1,0,1,1,0,9
6,3,4,2,0,9,1,1,0,0,1,1,11
7,0,0,1,0,1,0,0,0,1,0,0,2
8,3,3,1,1,8,1,1,1,1,1,1,12
9,3,0,0,1,4,1,0,0,1,0,0,5
